In [2]:
import numpy as np
import pandas as pd

import astropy

from astropy.timeseries import TimeSeries
from astropy.table import Table
from astropy import units as u
import plotly.graph_objects as go

import lightkurve as lk

from senior experience paper: KOI-4813, KOI-0521, KOI-5869, and KOI-2453

In [3]:
search_result_lc = lk.search_lightcurve('KOI-4813')
lc_collection = search_result_lc[search_result_lc.author == "Kepler"].download_all()
# want short exposure times https://www.ut.edu/uploadedFiles/Academics/Acta_Spartae/AS_0601p1Argentieri.pdf

In [4]:
def lightcurve_data(data):
    SAP=[]
    PDCSAP=[]
    norm=[]
    time=[]

    for x in range(0, len(data)-1):
        sap_ind=data[x]['sap_flux'].to_value().tolist()
        pdcsap_ind=data[x]['flux'].to_value().tolist() # source of the flux in the table is PDCSAP FLUX
        norm_ind=data[x].normalize()['flux'].to_value().tolist()
        time_ind=data[x]['time'].to_value('bkjd').tolist()

        SAP.append(sap_ind)
        PDCSAP.append(pdcsap_ind)
        norm.append(norm_ind)
        time.append(time_ind)
    
    stitched=lc_collection.stitch().flatten(window_length=901).remove_outliers(sigma=20, sigma_upper=4)
    fully_norm_flux=stitched['flux'].to_value().tolist() 
    full_time=stitched['time'].to_value('bkjd').tolist()
    return(PDCSAP, SAP, norm, time, fully_norm_flux, full_time) 


Simple Aperture Photometry = SAP

Pre-search Data Conditioning SAP flux = PDCSAP

More info can be found [here](https://heasarc.gsfc.nasa.gov/docs/tess/LightCurveFile-Object-Tutorial.html)

In [5]:
test1=lightcurve_data(lc_collection)

In [6]:
test1[5]

[131.51240196308936,
 131.5328366199028,
 131.55327117648994,
 131.57370563306904,
 131.59414018965617,
 131.61457484612765,
 131.6350093024812,
 131.6554438587118,
 131.67587851495045,
 131.69631307106465,
 131.71674752718536,
 131.73718208319042,
 131.75761673907255,
 131.778051194844,
 131.79848575061624,
 131.81892040626553,
 131.83935486192058,
 131.85978941745998,
 131.88022397287568,
 131.90065852829866,
 131.92109308360523,
 131.9415276387881,
 131.96196219386184,
 131.98239674893557,
 132.00283130400203,
 132.02326575884217,
 132.04370041368384,
 132.06413496840105,
 132.08456942312478,
 132.10500397773285,
 132.125438632218,
 132.14587308659247,
 132.16630764096772,
 132.18674229522003,
 132.20717684947886,
 132.2276113036205,
 132.2480458576465,
 132.26848051154957,
 132.2889149654584,
 132.30934951924428,
 132.3297841729218,
 132.35021862659778,
 132.37065318015811,
 132.39108773359476,
 132.41152228703868,
 132.4319568403589,
 132.45239139357,
 132.47282604678185,
 132.493

In [7]:
len(test1)

6

In [8]:
def lightcurve_traces(data):
    upper_range=len(data)-1

    data2plot=lightcurve_data(data)

    SAP_traces={}
    PDCSAP_traces={}
    norm_traces={}
    
    fully_norm_flux_trace=go.Scattergl(x=data2plot[5], y=data2plot[4])

    for x in range(0, upper_range):
        SAP_traces['SAP_trace_' + str(x)]=go.Scattergl(x=data2plot[3][x], name=f"Quarter {lc_collection[x].quarter}", y=data2plot[1][x])
        PDCSAP_traces['SAP_trace_' + str(x)]=go.Scattergl(x=data2plot[3][x], name=f"Quarter {lc_collection[x].quarter}", y=data2plot[0][x])
        norm_traces['SAP_trace_' + str(x)]=go.Scattergl(x=data2plot[3][x], name=f"Quarter {lc_collection[x].quarter}", y=data2plot[2][x])
    return(PDCSAP_traces, SAP_traces, norm_traces, fully_norm_flux_trace)


In [9]:
test2=lightcurve_traces(lc_collection)

In [10]:
len(test2[0])

13

In [16]:
def lightcurve_plot(data):
    
    traces2plot=lightcurve_traces(data)

    flux_unit=data[0]['flux'].unit.to_string()
    time_unit=data[0]['time'].format

    lc_name=data[0].meta['OBJECT']

    t_ini_limit=float(data[0].meta['TSTART'])
    t_fin_limit=float(data[len(data)-1].meta['TSTOP'])

    PDCSAP_fig_data, SAP_fig_data=list(traces2plot[0].values()), list(traces2plot[1].values())
    norm_fig_data, fully_norm_flux_fig_data=list(traces2plot[2].values()),traces2plot[3]
    
    layout=go.Layout(xaxis=dict(range=[t_ini_limit-20, t_fin_limit+20], autorange=False, zeroline=False),
                  hovermode=False)
    
    PDCSAP_fig, SAP_fig=go.Figure(PDCSAP_fig_data, layout), go.Figure(SAP_fig_data, layout)
    norm_fig, fully_norm_flux_fig=go.Figure(norm_fig_data, layout), go.Figure(fully_norm_flux_fig_data, layout)
    
    PDCSAP_fig.update_layout(title={
        'text': f"{lc_name}'s PDCSAP Lightcurve",
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'}, yaxis_title=f"PDCSAP Flux [{flux_unit}]",
                              xaxis_title=f" days [{time_unit}]", legend_title="Observation Quarters")
    
    SAP_fig.update_layout(title={
        'text': f"{lc_name}'s SAP Lightcurve",
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'}, yaxis_title=f"PDCSAP Flux [{flux_unit}]",
                              xaxis_title=f" days [{time_unit}]", legend_title="Observation Quarters")
    
    norm_fig.update_layout(title={
        'text': f"{lc_name}'s Normalized Lightcurve",
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'}, yaxis_title=f"Normalized Flux",
                              xaxis_title=f" days [{time_unit}]", legend_title="Observation Quarters")
    
    fully_norm_flux_fig.update_layout(title={
        'text': f"{lc_name}'s Normalized and Flattened Lightcurve",
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'}, yaxis_title=f"Normalized Flux",
                              xaxis_title=f" days [{time_unit}]")
    
    return(PDCSAP_fig, SAP_fig, norm_fig, fully_norm_flux_fig)

In [17]:
test3=lightcurve_plot(lc_collection)

In [18]:
test3[3].show()